## Importing Required Libraries

In [1]:
from keras.layers import Lambda, Input, Dense, Reshape, Dropout
from keras.utils import plot_model

from keras import backend as K
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import csv
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint




# Data generation

##Dataset for one parameter estimation is generated using below equation where we consider different random values of k and we get temperature values at 10 different points along the length of slab

$$T(x) = q(\frac{L-x}{k} + \frac{1}{h}) + T_{surr}$$

with other parameter values $q = 1000$, material length $L = 1m$, $h =12.5 $ and $T_{surr} = 298$ and $x$ is taken randomly on length.

In [2]:
np.random.seed(42)
k_values = np.random.rand(1000) * 190 + 10
x = np.linspace(0.0,0.9,10)
t_values = []
for k in k_values:
    temp = 1000.0*((1-x)/k+1.0/12.5)+298
    t_values.append(temp)
    
column_labels = [f'x{i+1}' for i in range(10)]
data = pd.DataFrame(t_values, columns=column_labels)
data['k'] = k_values
print(data)




             x1          x2          x3          x4          x5          x6  \
0    390.320942  389.088848  387.856754  386.624660  385.392565  384.160471   
1    383.245607  382.721046  382.196485  381.671925  381.147364  380.622803   
2    384.707860  384.037074  383.366288  382.695502  382.024716  381.353930   
3    386.081127  385.273015  384.464902  383.656789  382.848676  382.040564   
4    403.224790  400.702311  398.179832  395.657353  393.134874  390.612395   
..          ...         ...         ...         ...         ...         ...   
995  414.495559  410.846003  407.196447  403.546891  399.897336  396.247780   
996  383.426243  382.883619  382.340994  381.798370  381.255746  380.713121   
997  405.781220  403.003098  400.224976  397.446854  394.668732  391.890610   
998  383.248101  382.723291  382.198481  381.673671  381.148861  380.624051   
999  388.555082  387.499573  386.444065  385.388557  384.333049  383.277541   

             x7          x8          x9         x10

## Splitting data into independent and dependent variables

In [3]:
u = data.iloc[:,:10]
k = data.iloc[:,-1]

k.head(),u.head()

(0     81.162623
 1    190.635718
 2    149.078849
 3    123.745112
 4     39.643542
 Name: k, dtype: float64,
            x1          x2          x3          x4          x5          x6  \
 0  390.320942  389.088848  387.856754  386.624660  385.392565  384.160471   
 1  383.245607  382.721046  382.196485  381.671925  381.147364  380.622803   
 2  384.707860  384.037074  383.366288  382.695502  382.024716  381.353930   
 3  386.081127  385.273015  384.464902  383.656789  382.848676  382.040564   
 4  403.224790  400.702311  398.179832  395.657353  393.134874  390.612395   
 
            x7          x8          x9         x10  
 0  382.928377  381.696283  380.464188  379.232094  
 1  380.098243  379.573682  379.049121  378.524561  
 2  380.683144  380.012358  379.341572  378.670786  
 3  381.232451  380.424338  379.616225  378.808113  
 4  388.089916  385.567437  383.044958  380.522479  )

In [4]:
k = data.iloc[:,-1]
k.head()

0     81.162623
1    190.635718
2    149.078849
3    123.745112
4     39.643542
Name: k, dtype: float64

## Adding Noise in the data

## Adjust the value of noise standard deviation as needed for experimentation

In [5]:
max_value = (abs(u).max()).max()
var_e = 0.01*max_value
mu_e = 0

# Add random noise to the DataFrame
noise = np.random.normal(mu_e, var_e, size=u.shape)
u_noisy = u + noise
data_noisy = pd.concat([u_noisy, k], axis=1)
data_noisy.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,k
0,391.155979,382.813917,389.643346,389.493870,388.023083,389.239179,386.847069,383.854021,380.134472,371.427055,81.162623
1,385.264430,383.696993,383.472664,375.672342,376.067360,385.571685,379.912369,382.776129,379.182193,378.664388,190.635718
2,389.116959,381.612123,383.817970,380.523219,379.982970,379.901096,381.726975,377.762665,385.242509,374.466927,149.078849
3,385.202997,383.206670,391.264411,384.580422,387.697428,375.059751,382.487350,384.604812,380.002887,383.814923,123.745112
4,400.793994,407.324990,408.982608,393.952334,391.041408,397.442009,395.512500,383.110460,381.070454,379.198342,39.643542


## Standardising the combined Dataframe

In [6]:
# Create a StandardScaler instance

scaler = StandardScaler()
# Normalize the DataFrame and create a new DataFrame
data_noisy_standardised = pd.DataFrame(scaler.fit_transform(data_noisy), columns= data_noisy.columns)

print("\nNormalized DataFrame:")
print(data_noisy_standardised)


Normalized DataFrame:
           x1        x2        x3        x4        x5        x6        x7  \
0   -0.190683 -0.663958 -0.092750  0.038555  0.023590  0.348039  0.296801   
1   -0.543495 -0.605029 -0.554814 -1.121885 -1.092274 -0.044923 -0.584521   
2   -0.312788 -0.744155 -0.528957 -0.714611 -0.726818 -0.652511 -0.353905   
3   -0.547174 -0.637749  0.028636 -0.373972 -0.006804 -1.171247 -0.257270   
4    0.386484  0.971691  1.355383  0.412883  0.305300  1.226950  1.398078   
..        ...       ...       ...       ...       ...       ...       ...   
995  0.765099  1.312237  1.039318  1.006079  0.700104 -0.528595  1.928476   
996 -0.380069 -0.614331 -0.930329 -0.762297  0.127994 -0.602244 -0.660247   
997  0.776734  1.028090 -0.091575  0.262392  0.389146  0.630380  0.498095   
998 -0.401371 -0.534085 -0.762765  0.013590  0.064615 -0.105338 -2.363464   
999  0.001791  0.143652 -0.023140  0.225980  0.553775 -0.181699 -1.229930   

           x8        x9       x10         k  
0    0

In [7]:
# Create a Standard Scaler  instance
scaler1 = StandardScaler()

# Standardize the DataFrame and create a new DataFrame
data_standardised = pd.DataFrame(scaler1.fit_transform(data), columns=data.columns)

print("\nNormalized DataFrame:")
print(data_standardised)


Normalized DataFrame:
           x1        x2        x3        x4        x5        x6        x7  \
0   -0.247455 -0.247455 -0.247455 -0.247455 -0.247455 -0.247455 -0.247455   
1   -0.693879 -0.693879 -0.693879 -0.693879 -0.693879 -0.693879 -0.693879   
2   -0.601617 -0.601617 -0.601617 -0.601617 -0.601617 -0.601617 -0.601617   
3   -0.514970 -0.514970 -0.514970 -0.514970 -0.514970 -0.514970 -0.514970   
4    0.566723  0.566723  0.566723  0.566723  0.566723  0.566723  0.566723   
..        ...       ...       ...       ...       ...       ...       ...   
995  1.277861  1.277861  1.277861  1.277861  1.277861  1.277861  1.277861   
996 -0.682481 -0.682481 -0.682481 -0.682481 -0.682481 -0.682481 -0.682481   
997  0.728023  0.728023  0.728023  0.728023  0.728023  0.728023  0.728023   
998 -0.693721 -0.693721 -0.693721 -0.693721 -0.693721 -0.693721 -0.693721   
999 -0.358873 -0.358873 -0.358873 -0.358873 -0.358873 -0.358873 -0.358873   

           x8        x9       x10         k  
0   -0

## Splitting the dataset into train and test

## Ratio for Train-Test Split can be adjusted

In [8]:
u_train_noisy = data_noisy_standardised.iloc[:800,:-1]
u_test_noisy = data_noisy_standardised.iloc[800:,:-1]
u_train_noisy.shape, u_test_noisy.shape


((800, 10), (200, 10))

In [9]:
u_train = data_standardised.iloc[:800,:-1]
u_test = data_standardised.iloc[800:,:-1]
u_train.shape, u_test.shape

k_train = data_standardised.iloc[:800,-1]
k_test = data_standardised.iloc[800:,-1]
k_train.shape, k_test.shape

((800,), (200,))

In [10]:
shift_train = k_train*0.3
shift_test = k_test*0.3
k_train_prior = k_train + shift_train
k_test_prior = k_test + shift_test

#**VAE**

## Reparameterization Trick Function

$\mu_z + \epsilon \sigma_z$

In [11]:
# reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

## KL-Divergence Terms

KL-divergence loss = $\ln\frac{|\Gamma_{\text{pr}}|}{|\Gamma_{\text{post}}|} + \text{tr}(\Gamma_{\text{pr}}^{-1}\Gamma_{\text{post}}^{(m)}) + \|\mu_{\text{post}}^{(m)} - \mu_{\text{pr}}\|^2_{{\Gamma}_{pr}}$

$(\cdot)_p$ stands for prior and $(\cdot)_q$ stands for posterior

In [12]:
def kl_divergence_multivariate_normal_keras(mu_p, log_var_p, mu_q, log_var_q):
    sigma_p = K.exp(log_var_p)            ## log variance to variance
    sigma_q = K.exp(log_var_q)

    term1 = K.sum(log_var_p - log_var_q)

    term2 = K.sum(sigma_q/sigma_p)

    term3 = K.sum(K.square(mu_q - mu_p) / sigma_p)

    return (term1 + term2 + term3)

In [13]:
def js_divergence_multivariate_normal_keras(mu_post, log_var_post, inputs_k, js_param):
    var_post = K.exp(log_var_post)            ## log variance to variance

    term1 = K.sum(log_var_post)
    term2 = K.sum(K.square(mu_post - inputs_k) / var_post)

    return js_param*(term1 + term2)

In [14]:
import keras_tuner

def build_model(hp):
    
    input_shape_k = (1, )
    input_shape_x = (10,)
    latent_dim = 1
    
    intermediate_dim_1 = 8 # 1st layer
    intermediate_dim_2 = 5 # 2nd Layer
    intermediate_dim_3 = 3 # 3rd Layer

    inputs_k = Input(shape=input_shape_k, name='ground_truth')
    inputs_k_pr = Input(shape=input_shape_k, name = 'prior mean')
    inputs_x = Input(shape=input_shape_x, name='temp_input')
    inter_x1 = Dense(intermediate_dim_1, activation='tanh', name='encoder_intermediate_1')(inputs_x)
    inter_x2 = Dense(intermediate_dim_2, activation='tanh', name='encoder_intermediate_2')(inter_x1)
    inter_x3 = Dense(intermediate_dim_3, activation='tanh', name='encoder_intermediate_3')(inter_x2)

# q(z|x)
    z_mean = Dense(latent_dim, name='z_mean')(inter_x3)
    z_log_var = Dense(latent_dim, name='z_log_var')(inter_x3)

# use reparameterization trick to push the sampling out as input
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
    encoder = Model([inputs_x,inputs_k,inputs_k_pr], [z_mean, z_log_var,z], name='encoder')


    model_path = r'C:\Users\rajes\Desktop\UQ_VAE - datasets and codes\trained_decoder/best_model_2.h5'  # Update this path as necessary
    physics_based_decoder = load_model(model_path, compile=False)

# Freeze the weights of the physics-based decoder
    for layer in physics_based_decoder.layers:
        layer.trainable = False

    outputs = physics_based_decoder(z)  # Connect encoder output 'z' to the physics-based decoder

    decoder = Model(z, outputs, name='decoder')



    vae = Model([inputs_x,inputs_k,inputs_k_pr], outputs, name='vae_mlp')

    
# ##Customize Lost Function of the VAE Mod
    
    alpha_js = hp.Choice('alpha_js',[0.00001,0.001,0.1,0.5])
    js_param = (1.0-alpha_js)/alpha_js
 
    reconstruction_los = mse(inputs_x,outputs)
    

    kl_loss = (1.0/800.0)*kl_divergence_multivariate_normal_keras(inputs_k_pr, K.constant([0.0]), z_mean, z_log_var)
    label_loss = (1.0/800.0)*js_divergence_multivariate_normal_keras(z_mean,z_log_var,inputs_k,js_param)

    vae_loss =  (reconstruction_los +kl_loss + label_loss)
    

    
    #alpha_js = hp.Choice('alpha_js', [1e-2, 1e-3, 1e-4, 1e-5, 1e-6])
    vae.add_loss(vae_loss)
    optimizer = Adam(learning_rate= 0.001)  # Adjust the learning rate value as needed
    vae.compile(optimizer=optimizer)

    return vae

In [15]:

tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    project_name = 'hp_tuning')

In [16]:
tuner.search([u_train_noisy,k_train, k_train_prior],u_train_noisy, epochs=10, validation_data=([u_test_noisy,k_test, k_test_prior],u_test_noisy))

Trial 4 Complete [00h 00m 06s]
val_loss: -27.381105422973633

Best val_loss So Far: -3226.879638671875
Total elapsed time: 00h 00m 47s


In [17]:
best_models = tuner.get_best_models(num_models=1)

In [18]:
tuner.get_best_hyperparameters()[0].values 

{'alpha_js': 1e-05}